In [146]:
import pandas as pd
import numpy as np
df=pd.read_csv('updatedresultsdb.csv')

In [8]:
df_missing = df[(df['category'].isna()) | (df['description'].isna())]
df_missing.nunique()

Book-Title             1418
Book-Author             856
User-ID                4859
ISBN                   1465
Book-Rating              10
Year-Of-Publication      43
Publisher               278
Location               2727
Age                      88
category                 23
description               7
amazon_category           3
amazon_summary            1
GoodreadsDesc             0
GoodreadsGenre            0
new_category              3
new_description           1
dtype: int64

In [9]:
df_unique_books=pd.read_csv('newupdates.csv')

In [10]:
# Ensure keys are aligned (if needed)
df_unique_books.set_index(["Book-Title", "Book-Author"], inplace=True)
df.set_index(["Book-Title", "Book-Author"], inplace=True)

# Fill missing values in df using df_unique_books
df["description"] = df["description"].fillna(df_unique_books["DBAbstract"])
df["category"] = df["category"].fillna(df_unique_books["DBGenre"])

# Reset index if needed
df.reset_index(inplace=True)
df_unique_books.reset_index(inplace=True)

In [12]:
df["category"] = df["category"].replace("No genre found", np.nan)
df["description"] = df["description"].replace("No abstract found", np.nan)

In [23]:
df[(df['category'].isna()) | (df['description'].isna())].nunique()

Book-Title             1380
Book-Author             842
User-ID                4806
ISBN                   1426
Book-Rating              10
Year-Of-Publication      43
Publisher               274
Location               2705
Age                      88
category                 23
description               6
amazon_category           3
amazon_summary            0
GoodreadsDesc             0
GoodreadsGenre            0
new_category              3
new_description           0
dtype: int64

In [16]:
#Let's save this updated df
df.to_csv('DataEnrichmentDBGRAZ.csv', index=False)

In [137]:
df=pd.read_csv('DataEnrichmentDBGRAZ.csv')

In [138]:
df['description'].nunique()

5007

# USING WIKIPEDIA TO ENHANCE THE BOOK SUMMARIES FOR THE ONCE WE HAVE NO SUMMARIES!!!!


In [139]:
df_missing = df[(df['category'].isna()) | (df['description'].isna())]
df_missing.nunique()

Book-Title             1380
Book-Author             842
User-ID                4806
ISBN                   1426
Book-Rating              10
Year-Of-Publication      43
Publisher               274
Location               2705
Age                      88
category                 23
description               6
amazon_category           3
amazon_summary            0
GoodreadsDesc             0
GoodreadsGenre            0
new_category              3
new_description           0
dtype: int64

In [140]:
df_unique_books = df_missing.drop_duplicates(subset=['Book-Title', 'Book-Author'])

In [141]:
df_unique_books

,Book-Title,Book-Author,User-ID,ISBN,Book-Rating,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
3,"The Fires of Heaven (The Wheel of Time, Book 5)",Robert Jordan,276964,0812550307,10,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Stronghold (Dragon Star, Book 1)",Melanie Rawn,276964,0886774829,8,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Skybowl (Dragon Star, Book 3)",Melanie Rawn,276964,0886775957,10,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Running with Scissors,Augusten Burroughs,277042,0312283709,8,2002,St. Martin's Press,"hiram, georgia, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Roses Are Red (Alex Cross Novels),James Patterson,277042,0446605484,7,2001,Warner Vision,"hiram, georgia, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45662,Penguin Dreams and Stranger Things (A Bloom County Book),Berke Breathed,145449,0316107255,10,1985,Little Brown &amp; Co,"virginia beach, virginia, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47671,"Hello Darling, Are You Working?",Rupert Everett,152645,038072152X,6,1998,Perennial,"chicago, illinois, usa",28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
48088,Men and the Girls,Joanna Trollope,154345,0380724081,7,1995,Perennial,"white plains, new york, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52896,Almost Adam: A Novel,Petru Popescu,170513,0688148638,10,1996,Harpercollins,"butler, pennsylvania, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [103]:
# !pip install fuzzywuzzy
import wikipedia
import pandas as pd
import time
from fuzzywuzzy import fuzz  # For fuzzy matching

# Set Wikipedia language (optional)
wikipedia.set_lang("en")

# Function to get Wikipedia summary with verification
def get_wikipedia_summary(page_title, author_name, counter, total):
    try:
        print(f"Processing {counter}/{total}: '{page_title}' by '{author_name}'...")
        summary = wikipedia.summary(page_title)
        
        # Verify the author's name using fuzzy matching
        if fuzz.partial_ratio(author_name.lower(), summary.lower()) > 80:  # Adjust threshold as needed
            print(f"Summary found and verified for: '{page_title}'.")
            return summary
        else:
            print(f"Author '{author_name}' not found in the summary for: '{page_title}'. Skipping.")
            return None  # Return None if the author's name is not found
    except wikipedia.exceptions.PageError:
        print(f"No Wikipedia page found for: '{page_title}'. Skipping.")
        return None  # Return None if the page doesn't exist
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for: '{page_title}'. Skipping.")
        return None  # Return None for disambiguation errors

# Function to add a delay between requests
def fetch_summary_with_delay(page_title, author_name, counter, total):
    time.sleep(1)  
    return get_wikipedia_summary(page_title, author_name, counter, total)


# Apply the function to the DataFrame with a counter
total_books = len(df_unique_books)  # Total number of books in the DataFrame
results = []

for index, row in df_unique_books.iterrows():
    counter = index + 1  # Counter starts at 1
    summary = fetch_summary_with_delay(row['Book-Title'], row['Book-Author'], counter, total_books)
    results.append(summary)

# Add the results to the DataFrame
df_unique_books['wikipedia_summary'] = results

# Save the updated DataFrame to a new CSV file (optional)
# df_unique_books.to_csv('wikiunique.csv', index=False)


Processing 4/1387: 'The Fires of Heaven (The Wheel of Time, Book 5)' by 'Robert Jordan'...
Author 'Robert Jordan' not found in the summary for: 'The Fires of Heaven (The Wheel of Time, Book 5)'. Skipping.
Processing 8/1387: 'Stronghold (Dragon Star, Book 1)' by 'Melanie Rawn'...
Author 'Melanie Rawn' not found in the summary for: 'Stronghold (Dragon Star, Book 1)'. Skipping.
Processing 10/1387: 'Skybowl (Dragon Star, Book 3)' by 'Melanie Rawn'...
No Wikipedia page found for: 'Skybowl (Dragon Star, Book 3)'. Skipping.
Processing 11/1387: 'Running with Scissors' by 'Augusten Burroughs'...
Author 'Augusten Burroughs' not found in the summary for: 'Running with Scissors'. Skipping.
Processing 12/1387: 'Roses Are Red (Alex Cross Novels)' by 'James Patterson'...
Summary found and verified for: 'Roses Are Red (Alex Cross Novels)'.
Processing 17/1387: 'The Tin Collectors : A Novel (A Shane Scully Novel)' by 'Stephen J. Cannell'...
Summary found and verified for: 'The Tin Collectors : A Novel (

/vol/tensusers/nkalra/env/lib/python3.10/site-packages/wikipedia/wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /vol/tensusers/nkalra/env/lib/python3.10/site-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


Disambiguation error for: 'Prey'. Skipping.
Processing 190/1387: 'The Switch' by 'Sandra Brown'...
Author 'Sandra Brown' not found in the summary for: 'The Switch'. Skipping.
Processing 195/1387: 'Italian Neighbors Or, a Lapsed Anglo-Saxon in Verona' by 'Tim Parks'...
No Wikipedia page found for: 'Italian Neighbors Or, a Lapsed Anglo-Saxon in Verona'. Skipping.
Processing 196/1387: 'Mama Makes Up Her Mind: And Other Dangers of Southern Living' by 'Bailey White'...
Summary found and verified for: 'Mama Makes Up Her Mind: And Other Dangers of Southern Living'.
Processing 198/1387: 'Sleeping at the Starlite Motel: And Other Adventures on the Way Back Home' by 'Bailey White'...
Summary found and verified for: 'Sleeping at the Starlite Motel: And Other Adventures on the Way Back Home'.
Processing 201/1387: 'Bel Canto: A Novel' by 'Ann Patchett'...
Author 'Ann Patchett' not found in the summary for: 'Bel Canto: A Novel'. Skipping.
Processing 209/1387: 'Welcome to Temptation' by 'Jennifer Cru

/tmp/ipykernel_3155830/1348973006.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique_books['wikipedia_summary'] = results


In [108]:
df_unique_books=pd.read_csv('wikiunique.csv')

df_unique_books.set_index(["Book-Title", "Book-Author"], inplace=True)
df.set_index(["Book-Title", "Book-Author"], inplace=True)

# Fill missing values in df using df_unique_books
df["description"] = df["description"].fillna(df_unique_books["wikipedia_summary"])


# Reset index if needed
df.reset_index(inplace=True)
df_unique_books.reset_index(inplace=True)

In [121]:
# Replace None with NaN in the 'description' column
df["description"] = df["description"].replace(to_replace=[None], value=np.nan)

In [150]:
# df.to_csv('finalupdates.csv', index=False)
df=pd.read_csv('finalupdates.csv')

In [144]:
df[df['description'].isna()]

,Book-Title,Book-Author,User-ID,ISBN,Book-Rating,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
3,"The Fires of Heaven (The Wheel of Time, Book 5)",Robert Jordan,276964,0812550307,10,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Stronghold (Dragon Star, Book 1)",Melanie Rawn,276964,0886774829,8,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Skybowl (Dragon Star, Book 3)",Melanie Rawn,276964,0886775957,10,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Running with Scissors,Augusten Burroughs,277042,0312283709,8,2002,St. Martin's Press,"hiram, georgia, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
90,Chicks in Chainmail,Esther Friesner,277427,0671876821,8,1995,Baen,"gilbert, arizona, usa",48.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87208,Isabel's Bed,Elinor Lipman,276670,0671015648,6,1998,Washington Square Press,"toronto, ontario, canada",30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87222,The Lovely Bones: A Novel,Alice Sebold,276680,0316666343,7,2002,"Little, Brown","hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87228,The Emperor of Ocean Park (Vintage Contemporaries (Paperback)),Stephen L. Carter,276680,0375712925,7,2003,Vintage Books USA,"hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87236,The Probable Future,ALICE HOFFMAN,276680,0385507607,6,2003,Doubleday,"hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [145]:
df['description'].nunique()

5234

In [151]:
df[(df['category'].isna()) | (df['description'].isna())]

,Book-Title,Book-Author,User-ID,ISBN,Book-Rating,Year-Of-Publication,Publisher,Location,Age,category,description,amazon_category,amazon_summary,GoodreadsDesc,GoodreadsGenre,new_category,new_description
3,"The Fires of Heaven (The Wheel of Time, Book 5)",Robert Jordan,276964,0812550307,10,1994,Tor Fantasy,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,"Stronghold (Dragon Star, Book 1)",Melanie Rawn,276964,0886774829,8,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,"Skybowl (Dragon Star, Book 3)",Melanie Rawn,276964,0886775957,10,1994,Daw Books,"villa ridge, missouri, usa",34.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,Running with Scissors,Augusten Burroughs,277042,0312283709,8,2002,St. Martin's Press,"hiram, georgia, usa",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11,Roses Are Red (Alex Cross Novels),James Patterson,277042,0446605484,7,2001,Warner Vision,"hiram, georgia, usa",NaN,NaN,"Alex Cross is a crime, mystery, and thriller novel series written by James Patterson. The protagonist of the series is Alex Cross, an African-American Metropolitan Police Department detective and father who counters threats to his family and to the city of Washington, D.C. Supporting characters include two of Cross's children, Damon and Janelle, as well as his grandmother Nana Mama. The series is usually narrated in first-person perspective by Alex Cross, and occasionally from the villains' point of view in third-person.\nThe series began in 1993 and is ongoing. Nearly all the stories have made bestsellers lists and garnered favorable reviews, especially Double Cross; Cross Fire; I, Alex Cross; and Alex Cross, Run.\nThe series is published worldwide, with Little, Brown currently holding publication rights in the United States, in paperback, hardcover, and audiobook editions. The first book in the series, Along Came a Spider, was released in 1993 to positive reviews, spawning a series of over twenty subsequent novels. The series has resulted in three movies, Kiss the Girls (1997), Along Came a Spider (2001), and Alex Cross (2012).\n\n",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87222,The Lovely Bones: A Novel,Alice Sebold,276680,0316666343,7,2002,"Little, Brown","hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87228,The Emperor of Ocean Park (Vintage Contemporaries (Paperback)),Stephen L. Carter,276680,0375712925,7,2003,Vintage Books USA,"hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87236,The Probable Future,ALICE HOFFMAN,276680,0385507607,6,2003,Doubleday,"hopewell junction, new york, usa",55.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87258,The Poisonwood Bible: A Novel,Barbara Kingsolver,276681,0060930535,9,1999,Perennial,"chicago, illinois, usa",43.0,NaN,"The Poisonwood Bible (1998), by Barbara Kingsolver, is a best-selling novel about a missionary family, the Prices, who in 1959 move from the U.S. state of Georgia to the village of Kilanga in the Belgian Congo, close to the Kwilu River. \nThe novel's title refers to Bible errata. The father of the family creates his own ""misprint"" of the Bible. He concludes his sermons with the Kikongo expression ""Tata Jesus is bängala"" with the intent of saying ""Jesus is most precious"". In his hurried mispronunciation, he actually says ""Jesus is poisonwood"".",NaN,NaN,NaN,NaN,NaN,NaN
